## Tiger Census
* https://www2.census.gov/geo/pdfs/maps-data/data/tiger/tgrshp2019/TGRSHP2019_TechDoc.pdf
* S1200 - secondary road
* S1100 - primary road
* S1400 - local roads
* Build off scripts/cut_road_segments.py


In [154]:
import datetime
import dask.dataframe as dd
import dask_geopandas as dg
from dask import delayed, compute

import geopandas as gpd
import pandas as pd
from calitp_data_analysis.sql import to_snakecase

import geopandas
from segment_speed_utils import helpers
from segment_speed_utils.project_vars import analysis_date
from shared_utils import dask_utils, geography_utils, utils

GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/"
SHARED_GCS = f"{GCS_FILE_PATH}shared_data/"

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### Tiger - Load Roads

In [3]:
def load_roads(road_type_wanted: list, buffer_or_not: bool = False):
    """
    Load some or all of the roads based on what you filter.
    Can also buffer the roads or not.

    Args:
        road_type_wanted (list): the type of roads you want.
        https://www2.census.gov/geo/pdfs/maps-data/data/tiger/tgrshp2019/TGRSHP2019_TechDoc.pdf

        buffer_or_not (bool): add a buffer of 200 or not

    Returns:
        GeoDataFrame. As of 4/18/23, returns 953914 nunique linearid
    """
    df = gpd.read_parquet(
        f"{SHARED_GCS}all_roads_2020_state06.parquet",
        filters=[("MTFCC", "in", road_type_wanted)],
        columns=["LINEARID", "geometry", "FULLNAME"],
    ).to_crs(geography_utils.CA_NAD83Albers)

    if buffer_or_not:
        df = df.assign(geometry=df.geometry.buffer(200))

    df = to_snakecase(df)

    return df

In [4]:
# local_primary_seconary_og_rds.linearid.nunique()

In [5]:
# type(local_primary_seconary_og_rds)

In [6]:
# Local roads that are sjoined to bus routes, not segmented
# local_roads = gpd.read_parquet(f'{SHARED_GCS}local_roads_gtfs_shapes_sjoin.parquet')

In [7]:
# local_roads.linearid.nunique()

In [8]:
# All primary/secondary roads regardless of instersection w bus routes
# Segmented
# primary_secondary = gpd.read_parquet(f'{SHARED_GCS}segmented_primary_secondary_roads.parquet')

In [9]:
# primary_secondary.linearid.nunique()

In [10]:
# local_primary_seconary_og_rds.linearid.nunique() - (primary_secondary.linearid.nunique() + local_roads.linearid.nunique())

In [11]:
# primary_secondary.plot()

In [12]:
# local_roads.plot()

In [13]:
# local_primary_seconary_og_rds.plot()

### GTFS Shapes

In [14]:
def gtfs_stops_operators(date) -> gpd.GeoDataFrame:
    """
    Load and merge stops
    with trips to get operator and
    feed key information.

    Args:
        date: date wanted for the datasets to be drawn from

    Returns:
        GeoDataFrame
    """
    stops = (
        helpers.import_scheduled_stops(
            date, (), ["feed_key", "stop_id", "stop_key", "geometry"]
        )
        .compute()
        .drop_duplicates()
    )

    stops = stops.set_crs(geography_utils.CA_NAD83Albers)

    # Buffer
    stops = stops.assign(buffered_geometry=stops.geometry.buffer(50))

    # Set geometry
    stops = stops.set_geometry("buffered_geometry")

    # Merge for operator information
    trips = (
        helpers.import_scheduled_trips(analysis_date, (), ["name", "feed_key"])
        .compute()
        .drop_duplicates()
    )
    
    
    m1 = pd.merge(stops, trips, on=["feed_key"], how="left")

    # Fill in na
    m1.name = m1.name.fillna("None")

    return m1

In [15]:
# stops = gtfs_stops_operators(analysis_date)

In [16]:
def gtfs_routes_operators(date) -> gpd.GeoDataFrame:
    """
    Load and merge routes
    with trips to get operator and
    feed key information.

    Args:
        date: date wanted for the datasets to be drawn from

    Returns:
        GeoDataFrame
    """
    gtfs_shapes = helpers.import_scheduled_shapes(date).compute().drop_duplicates()
    
    gtfs_shapes = gtfs_shapes.set_crs(geography_utils.CA_NAD83Albers)
    
    trips = (
        helpers.import_scheduled_trips(date, (), ["name", "shape_array_key"])
        .compute()
        .drop_duplicates()
    )
    
    m1 = pd.merge(gtfs_shapes, trips, how="outer", on="shape_array_key")

    return m1

In [17]:
def order_operators(date) -> list:
    """
    Re order a list of operators so some of the largest
    ones will be at the top of the list.

    Args:
        date: date wanted for the datasets to be drawn from
    """
    operator_list = (
        helpers.import_scheduled_trips(date, (), ["name"]).compute().sort_values("name")
    )
    operator_list = operator_list.name.unique().tolist()

    # Reorder list so the biggest operators are at the beginning
    # based on NTD services data
    big_operators = [
        "LA DOT Schedule",
        "LA Metro Bus Schedule",
        "LA Metro Rail Schedule",
        "Bay Area 511 Muni Schedule",
        "Bay Area 511 AC Transit Schedule",
        "Bay Area 511 Santa Clara Transit Schedule",
        "Bay Area 511 BART Schedule",
        "San Diego Schedule",
        "OCTA Schedule",
        "Sacramento Schedule",
        "Bay Area 511 Sonoma-Marin Area Rail Transit Schedule",
        "Bay Area 511 SFO AirTrain Schedule",
        "Bay Area 511 South San Francisco Shuttle Schedule",
        "Bay Area 511 Marin Schedule",
        "Bay Area 511 County Connection Schedule",
        "Bay Area 511 MVGO Schedule",
        "Bay Area 511 Commute.org Schedule",
        "Bay Area 511 Union City Transit Schedule",
        "Bay Area 511 BART Schedule",
        "Bay Area 511 Caltrain Schedule",
        "Bay Area 511 Fairfield and Suisun Transit Schedule",
        "Bay Area 511 Dumbarton Express Schedule",
        "Bay Area 511 SamTrans Schedule",
        "Bay Area 511 Vine Transit Schedule",
        "Bay Area 511 Tri-Valley Wheels Schedule",
        "Bay Area 511 Sonoma County Transit Schedule",
        "Bay Area 511 Santa Rosa CityBus Schedule",
        "Bay Area 511 Golden Gate Transit Schedule",
        "Bay Area 511 Golden Gate Ferry Schedule",
        "Bay Area 511 San Francisco Bay Ferry Schedule",
        "Bay Area 511 SolTrans Schedule",
        "Bay Area 511 ACE Schedule",
        "Bay Area 511 Emery Go-Round Schedule",
        "Bay Area 511 Tri Delta Schedule",
        "Bay Area 511 Petaluma Schedule",
        "Bay Area 511 Capitol Corridor Schedule",
    ]

    # Delete off the big operators
    operator_list = list(set(operator_list) - set(big_operators))

    # Add back in the operators
    final_list = big_operators + operator_list

    return final_list

In [18]:
# op_list = order_operators(analysis_date)

### Tiger Local Roads

#### Cut all roads - stops 1st then routes 
* Use some small operators to test.

In [19]:
def loop_sjoin(date, local_roads_gdf, gdf_routes_stops):
    """
    gdf: stops or routes gdf
    """
    # Empty dataframe
    sjoin_full_results = pd.DataFrame()
    
    # Find all unique operators, ordered by largest operators first
    operators_list = order_operators(date)
    
    # Loop through and sjoin by operator
    for operator in operators_list:
        shapes_filtered = gdf_routes_stops.loc[
            gdf_routes_stops.name == operator
        ].reset_index(drop=True)

        # Delete any local road linear ids that have already been found by an operator
        try:
            # List of linear IDS
            linearid_to_delete = sjoin_full_results.linearid.unique().tolist()
            
            # Filter out the linear IDS in buffered local roads
            local_roads_gdf = local_roads_gdf[
                ~local_roads_gdf.linearid.isin(linearid_to_delete)
            ].reset_index(drop=True)
        except:
            pass

        sjoin1 = (
            gpd.sjoin(
                local_roads_gdf,
                shapes_filtered,
                how="inner",
                predicate="intersects",
            )[["linearid"]]
            .drop_duplicates()
            .reset_index(drop=True)
        )

        sjoin_full_results = pd.concat([sjoin_full_results, sjoin1], axis=0)

    sjoin_full_results = sjoin_full_results.drop_duplicates()
    
    return sjoin_full_results

In [20]:
def cut_stops1(date):
    start = datetime.datetime.now()
    
    # Load stops
    gtfs_stops = gtfs_stops_operators(date)
    
    # Load local roads - buffered
    local_roads_buffered = load_roads(["S1400"], True)

    # Load local roads - not buffered
    local_roads_og = load_roads(["S1400"], False)

    # Loop through and sjoin by operator
    stops_sjoin = loop_sjoin(date, local_roads_buffered, gtfs_stops)

    # Merge back to original local roads 
    m1 = pd.merge(local_roads_og, stops_sjoin, on="linearid", how="inner")

    # Fill in null values for fullname
    m1.fullname = m1.fullname.fillna("None")
    
    # Find linear ids to delete
    linearid_to_delete = m1.linearid.unique().tolist()
    
    # Save
    m1.to_parquet(f"{SHARED_GCS}local_roads_stops_sjoin.parquet")
    
    end = datetime.datetime.now()
    print(f"Done with sjoin with stops and local roads. Time lapsed: {end-start}")
    
    return m1, linearid_to_delete

In [21]:
def cut_routes1(date, linearid_to_filter:list):
    start = datetime.datetime.now()

    # Load stops
    gtfs_routes = gtfs_routes_operators(date)
    
    # Load local roads - buffered
    local_roads_buffered = load_roads(["S1400"], True)
    
    # Delete out linear ids that have already been found
    local_roads_buffered = local_roads_buffered[~local_roads_buffered.linearid.isin(linearid_to_filter)].reset_index(drop=True)
    
    # Load local roads - not buffered
    local_roads_og = load_roads(["S1400"], False)
    local_roads_og = local_roads_og[~local_roads_og.linearid.isin(linearid_to_filter)].reset_index(drop=True)
    
    # Loop through and sjoin by operator
    routes_sjoin = loop_sjoin(date, local_roads_buffered,gtfs_routes)
    
    # Merge back to original local roads 
    m1 = pd.merge(local_roads_og, routes_sjoin, on="linearid", how="inner")

    # Fill in null values for fullname
    m1.fullname = m1.fullname.fillna("None")
    
    # Save
    m1.to_parquet(f"{SHARED_GCS}local_roads_routes_sjoin.parquet")
    
    end = datetime.datetime.now()
    print(f"Done with sjoin with routes and local roads. Time lapsed: {end-start}")
    return m1

In [22]:
def cut_local_roads(date):
    start = datetime.datetime.now()
    print(start)
    
    # Deal with stops first
    stops_sjoin, linear_id_stops = cut_stops1(date)
    
    # Move onto routes
    routes_sjoin = cut_routes1(date, linear_id_stops)
    
    # Stack
    all_local_roads = pd.concat([stops_sjoin, routes_sjoin], axis=0)
    
    all_local_roads.to_parquet(f"{SHARED_GCS}local_roads_all_routes_stops_sjoin.parquet")
    end = datetime.datetime.now()
    
    print(f"Done with all local roads. Time lapsed: {end-start}")
    return all_local_roads

In [23]:
# all_ops = cut_local_roads(analysis_date)

2023-04-18 14:03:40.957812
Done with sjoin with stops and local roads. Time lapsed: 0:06:29.965441
Done with sjoin with routes and local roads. Time lapsed: 0:09:58.024634
Done with all local roads. Time lapsed: 0:17:14.536012


In [26]:
all_ops.shape

(322720, 3)

In [172]:
def chunk_df(gdf, chunk_row_size:int):
    # Specify how many rows I want the gdf to broken into per df
    n = chunk_row_size
    
    # Break it out
    list_df = [gdf[i:i+n] for i in range(0,gdf.shape[0],n)]
    
    # Turn each dataframe to a dask one
    my_ddfs = []
    for df in list_df:
        ddf = dd.from_pandas(df, npartitions= 1)
        my_ddfs.append(ddf)
    
    return my_ddfs, len(my_ddfs)

In [174]:
ddfs, length = chunk_df(all_ops, 10000)

In [176]:
length

33

In [175]:
len(ddfs)

33

In [181]:
my_results = []
for i in [*range(0,length)]:
    my_df = ddfs[i]
    cut_geometry = delayed(geography_utils.cut_segments)(my_df, ["linearid", "fullname"], 1_000)
    
    my_results.append(cut_geometry)

In [183]:
start = datetime.datetime.now()
print(start)
results = [compute(i)[0] for i in my_results] 

2023-04-18 16:36:43.202119


KeyboardInterrupt: 

In [ ]:
results_gdf = pd.concat(results)

In [ ]:
results_gdf.to_parquet(f"{SHARED_GCS}segmented_local_roads.parquet")

In [ ]:
end = datetime.datetime.now()
print(end)

### Concat local roads and primary/secondary ones

In [ ]:
def cut_primary_secondary_roads():
    start = datetime.datetime.now()
    print(start)

    # Find all primary and secondary roads
    # regardless of intersection w/ GTFS shapes
    primary_secondary_mtfcc = ["S1100", "S1200"]
    primary_secondary_roads = load_roads(primary_secondary_mtfcc)

    segments = geography_utils.cut_segments(
        primary_secondary_roads, ["linearid", "fullname"], 1_000  # 1 km segments
    )

    segments.to_parquet(f"{SHARED_GCS}segmented_primary_secondary_roads.parquet")

    end = datetime.datetime.now()
    print(f"time lapsed for cutting primary & secondary roads: {end-start}")
    return segments

In [ ]:
primary_secondary = cut_primary_secondary_roads()

In [ ]:
def cut_all_roads(date):
    start = datetime.datetime.now()

    # Segment primary and secondary roads
    segmented_primary_secondary_rds = cut_primary_secondary_roads()

    # Find local roads that intersect  with GTFS shapes, then
    # segment them
    local_roads_gdf = cut_local_roads(date)

    # Concat
    all_roads = pd.concat(
        [segmented_primary_secondary_rds, segmented_local_rds], axis=0
    )
    all_roads.to_parquet(f"{SHARED_GCS}segmented_all_roads.parquet")

    end = datetime.datetime.now()
    print(f"time lapsed: {end-start}")

In [ ]:
test_all_roads = cut_all_roads(analysis_date)

### Draft

In [ ]:
def cut_stops(date):
    start = datetime.datetime.now()
    print(start)
    
    # Load stops
    gtfs_stops = gtfs_stops_operators(date)
    
    # Load local roads - buffered
    local_roads_buffered = load_roads(["S1400"], True)

    # Load local roads - not buffered
    local_roads_og = load_roads(["S1400"], False)

    # Empty dataframe
    sjoin_full_results = pd.DataFrame()

    # Loop through and sjoin by operator
    for operator in ['TART, North Lake Tahoe Schedule','Eastern Sierra Schedule']:
        shapes_filtered = gtfs_stops.loc[
            gtfs_stops.name == operator
        ].reset_index(drop=True)

        # Delete any local road linear ids that have already been found by an operator
        try:
            # List of linear IDS
            linearid_to_delete = sjoin_full_results.linearid.unique().tolist()
            
            # Filter out the linear IDS in buffered local roads
            local_roads_buffered = local_roads_buffered[
                ~local_roads_buffered.linearid.isin(linearid_to_delete)
            ].reset_index(drop=True)
        except:
            pass

        sjoin1 = (
            gpd.sjoin(
                local_roads_buffered,
                shapes_filtered,
                how="inner",
                predicate="intersects",
            )[["linearid"]]
            .drop_duplicates()
            .reset_index(drop=True)
        )

        sjoin_full_results = pd.concat([sjoin_full_results, sjoin1], axis=0)

    sjoin_full_results = sjoin_full_results.drop_duplicates()

    # Merge back to original local roads 
    m1 = pd.merge(local_roads_og, sjoin_full_results, on="linearid", how="inner")

    # Fill in null values for fullname
    m1.fullname = m1.fullname.fillna("None")
    
    # Save
    # m1.to_parquet(f"{SHARED_GCS}local_roads_gtfs_shapes_sjoin.parquet")
    
    end = datetime.datetime.now()
    print(f"Done with sjoin with stops and local roads. Time lapsed: {end-start}")
    return m1, linearid_to_delete

In [ ]:
def cut_routes(date, linearid_to_filter:list):
    start = datetime.datetime.now()
    print(start)
    
    # Load stops
    gtfs_routes = gtfs_routes_operators(date)
    
    # Load local roads - buffered
    local_roads_buffered = load_roads(["S1400"], True)
    # Delete out linear ids that have already been found
    local_roads_buffered = local_roads_buffered[~local_roads_buffered.linearid.isin(linearid_to_filter)].reset_index(drop=True)
    
    # Load local roads - not buffered
    local_roads_og = load_roads(["S1400"], False)
    local_roads_og = local_roads_og[~local_roads_og.linearid.isin(linearid_to_filter)].reset_index(drop=True)

    # Find all unique operators, ordered by largest operators first
    # operators_list = order_operators(date)

    # Empty dataframe
    sjoin_full_results = pd.DataFrame()

    # Loop through and sjoin by operator
    for operator in ['TART, North Lake Tahoe Schedule','Eastern Sierra Schedule']:
        shapes_filtered = gtfs_routes.loc[
            gtfs_routes.name == operator
        ].reset_index(drop=True)

        # Delete any local road linear ids that have already been found by an operator
        try:
            # List of linear IDS
            linearid_to_delete = sjoin_full_results.linearid.unique().tolist()
            
            # Filter out the linear IDS in buffered local roads
            local_roads_buffered = local_roads_buffered[
                ~local_roads_buffered.linearid.isin(linearid_to_delete)
            ].reset_index(drop=True)
        except:
            pass

        sjoin1 = (
            gpd.sjoin(
                local_roads_buffered,
                shapes_filtered,
                how="inner",
                predicate="intersects",
            )[["linearid"]]
            .drop_duplicates()
            .reset_index(drop=True)
        )

        sjoin_full_results = pd.concat([sjoin_full_results, sjoin1], axis=0)

    sjoin_full_results = sjoin_full_results.drop_duplicates()

    # Merge back to original local roads 
    m1 = pd.merge(local_roads_og, sjoin_full_results, on="linearid", how="inner")

    # Fill in null values for fullname
    m1.fullname = m1.fullname.fillna("None")
    
    # Save
    # m1.to_parquet(f"{SHARED_GCS}local_roads_gtfs_shapes_sjoin.parquet")
    
    end = datetime.datetime.now()
    print(f"Done with sjoin with stops and local roads. Time lapsed: {end-start}")
    return m1

In [ ]:
"""
def join_primary_secondary(gtfs_shape_gdf):
    
    # Load secondary-primary roads
    primary_secondary_mtfcc = ['S1100','S1200']
    primary_secondary_roads = load_roads(primary_secondary_mtfcc)
    
    sjoin1 = gpd.sjoin(
        primary_secondary_roads,
        gtfs_shape_gdf,
        how = "inner",
        predicate = "intersects"
    ).drop_duplicates().reset_index(drop=True)
    
    # Save to GCS
    sjoin1.to_parquet(f'{SHARED_GCS}primary_secondary_roads_gtfs_shapes.parquet')
    print('Done with primary and secondary roads')
    return sjoin1
    """

#### Test w/ LA Metro first - 25 minutes long

In [ ]:
def la_metro(date):

    # Load Shapes
    gtfs_shape_gdf = gtfs_shapes_operators(date)

    # Load local roads - buffered
    local_roads_buffered = load_roads(["S1400"], True)

    # Load local roads - not buffered
    local_roads_og = load_roads(["S1400"], False)

    # Empty dataframe
    sjoin_full_results = pd.DataFrame()

    # Loop through and join by operator
    for operator in [
        "LA Metro Bus Schedule",
        "LA Metro Rail Schedule",
    ]:

        shapes_filtered = gtfs_shape_gdf.loc[
            gtfs_shape_gdf.name == operator
        ].reset_index(drop=True)

        # Delete any local road linearids that have already been found by an operator
        try:
            linearid_to_delete = sjoin_full_results.linearid.unique().tolist()
            local_roads_buffered = local_roads_buffered[
                ~local_roads_buffered.linearid.isin(linearid_to_delete)
            ].reset_index(drop=True)
        except:
            pass

        sjoin1 = (
            gpd.sjoin(
                local_roads_buffered,
                shapes_filtered,
                how="inner",
                predicate="intersects",
            )[["linearid"]]
            .drop_duplicates()
            .reset_index(drop=True)
        )

        sjoin_full_results = pd.concat([sjoin_full_results, sjoin1], axis=0)

    sjoin_full_results = sjoin_full_results.drop_duplicates()

    # Merge back to original local roads -> a GDF
    merge1 = pd.merge(local_roads_og, sjoin_full_results, on="linearid", how="inner")

    # Fill in null values for fullname
    merge1.fullname = merge1.fullname.fillna("None")

    # Save
    print("Done with LA Metro")

    return merge1

In [ ]:
# 2023-04-10 11:20:11.194365
# Done with LA Metro
# time lapsed: 0:25:57.435602
def cut_metro(date):
    start = datetime.datetime.now()
    print(start)

    # Find local roads that intersect  with LA Metro
    local_roads = la_metro(date)

    segments = geography_utils.cut_segments(
        local_roads, ["linearid", "fullname"], 1_000  # 1 km segments
    )

    end = datetime.datetime.now()
    print(f"time lapsed: {end-start}")

    return segments

In [ ]:
def gtfs_shapes_all_roads(date):

    start = datetime.datetime.now()
    print(start)

    # Find all primary and secondary roads
    # regardless of intersection
    primary_secondary_mtfcc = ["S1100", "S1200"]
    primary_secondary_roads = load_roads(primary_secondary_mtfcc)

    # Find local roads that intersect  with GTFS shapes
    local_roads_gdf = join_local_roads(date)

    # Concat
    all_roads = pd.concat([primary_secondary_roads, local_roads_gdf], axis=0)

    # Save
    all_roads.to_parquet(f"{SHARED_GCS}intersected_local_all_primary_sec_roads.parquet")

    """
    # Find segments
    segments = geography_utils.cut_segments(
        all_roads,
        ["linearid", "fullname"],
        1_000 # 1 km segments
    )
    
    segments.to_parquet(f"{SHARED_GCS}census_road_segments.parquet")
    """
    end = datetime.datetime.now()
    print(f"time lapsed: {end-start}")
    return all_roads

In [ ]:
def cut_local_roads(date):
    start = datetime.datetime.now()
    # Load Shapes
    gtfs_shape_gdf = gtfs_shapes_operators(date)

    # Load local roads - buffered
    local_roads_buffered = load_roads(["S1400"], True)

    # Load local roads - not buffered
    local_roads_og = load_roads(["S1400"], False)

    # Find all unique operators, ordered by
    # largest operators first
    operators_list = order_operators(date)

    # Empty dataframe
    sjoin_full_results = pd.DataFrame()

    # Loop through and join by operator
    for operator in operators_list:

        shapes_filtered = gtfs_shape_gdf.loc[
            gtfs_shape_gdf.name == operator
        ].reset_index(drop=True)

        # Delete any local road linearids that have already been found by an operator
        try:
            linearid_to_delete = sjoin_full_results.linearid.unique().tolist()
            local_roads_buffered = local_roads_buffered[
                ~local_roads_buffered.linearid.isin(linearid_to_delete)
            ].reset_index(drop=True)
        except:
            pass

        sjoin1 = (
            gpd.sjoin(
                local_roads_buffered,
                shapes_filtered,
                how="inner",
                predicate="intersects",
            )[["linearid"]]
            .drop_duplicates()
            .reset_index(drop=True)
        )

        sjoin_full_results = pd.concat([sjoin_full_results, sjoin1], axis=0)

    sjoin_full_results = sjoin_full_results.drop_duplicates()
    sjoin_full_results.to_parquet(f"{SHARED_GCS}local_roads_gtfs_shapes.parquet")

    # Merge back to original local roads -> a GDF
    merge1 = pd.merge(local_roads_og, sjoin_full_results, on="linearid", how="inner")

    # Fill in null values for fullname
    merge1.fullname = merge1.fullname.fillna("None")

    # Save
    merge1.to_parquet(f"{SHARED_GCS}local_roads_gtfs_shapes_sjoin.parquet")
    print("Done with sjoin local roads")

    end = datetime.datetime.now()
    print(f"time lapsed for cutting local roads: {end-start}")
    return segments